In [1]:
# pip install sktime

In [2]:
import pandas as pd
from scipy.stats import boxcox
from scipy.special import inv_boxcox
import numpy as np
from sktime.forecasting.arima import ARIMA
from sktime.forecasting.compose import TransformedTargetForecaster
from sktime.transformations.series.detrend import Deseasonalizer
from sktime.forecasting.trend import PolynomialTrendForecaster
from sktime.transformations.series.detrend import Detrender
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.base import ForecastingHorizon
from sklearn.metrics import mean_squared_error
from sktime.forecasting.compose import make_reduction
from sklearn.exceptions import ConvergenceWarning
import requests
from sklearn.ensemble import (
    HistGradientBoostingRegressor,
    GradientBoostingRegressor,
    RandomForestRegressor,
)
from sklearn.linear_model import ElasticNetCV
from sktime.forecasting.model_selection import (
    ForecastingGridSearchCV,
    ExpandingWindowSplitter,
)
from sktime.forecasting.compose import MultiplexForecaster
from sklearn.neighbors import KNeighborsRegressor
from sktime.forecasting.ets import AutoETS
from sktime.transformations.series.boxcox import LogTransformer


import warnings

In [3]:
from sklearn.preprocessing import StandardScaler
from sktime.forecasting.compose import ForecastingPipeline
from sktime.transformations.series.adapt import TabularToSeriesAdaptor


def initialize_arima_forecaster():
    pipe = ForecastingPipeline(
        steps=[
            ("standardize", TabularToSeriesAdaptor(StandardScaler())),
            (
                "forecaster",
                TransformedTargetForecaster(
                    [
                        ("log_transformer", LogTransformer()),
                        (
                            "deseasonalizer_daily",
                            Deseasonalizer(sp=24, model="additive"),
                        ),
                        (
                            "deseasonalizer_weekly",
                            Deseasonalizer(sp=24 * 7, model="additive"),
                        ),
                        ("residual_forecaster", ARIMA(1, 0, 1)),
                    ]
                ),
            ),
        ]
    )

    return pipe


def initialize_elasticnet_forecaster():
    deseasonalizer_weekly = Deseasonalizer(sp=24 * 7, model="additive")
    # Create the TransformedTargetForecaster pipeline
    pipe = ForecastingPipeline(
        steps=[
            ("standardize", TabularToSeriesAdaptor(StandardScaler())),
            (
                "forecaster",
                TransformedTargetForecaster(
                    [
                        ("log_transformer", LogTransformer()),
                        # ("deseasonalizer_weekly", Deseasonalizer(sp=24*7, model="additive")),
                        (
                            "forecast",
                            make_reduction(
                                ElasticNetCV(n_jobs=-1),
                                window_length=24,
                                strategy="direct",
                            ),
                        ),
                    ]
                ),
            ),
        ]
    )

    return pipe


def initialize_rf_forecaster():
    # Create the TransformedTargetForecaster pipeline
    pipe = ForecastingPipeline(
        steps=[
            ("standardize", TabularToSeriesAdaptor(StandardScaler())),
            (
                "forecaster",
                TransformedTargetForecaster(
                    [
                        ("log_transformer", LogTransformer()),
                        # ("deseasonalizer_weekly", Deseasonalizer(sp=24*7, model="additive")),
                        (
                            "forecast",
                            make_reduction(
                                RandomForestRegressor(n_estimators=200, n_jobs=-1),
                                window_length=24,
                                strategy="direct",
                            ),
                        ),
                    ]
                ),
            ),
        ]
    )

    return pipe


def initialize_gb_forecaster():
    deseasonalizer_daily = Deseasonalizer(sp=24, model="additive")
    pipe = ForecastingPipeline(
        steps=[
            ("standardize", TabularToSeriesAdaptor(StandardScaler())),
            (
                "forecaster",
                TransformedTargetForecaster(
                    [
                        ("log_transformer", LogTransformer()),
                        # ("deseasonalizer_weekly", Deseasonalizer(sp=24*7, model="additive")),
                        (
                            "forecast",
                            make_reduction(
                                GradientBoostingRegressor(n_estimators=200),
                                window_length=24,
                                strategy="direct",
                            ),
                        ),
                    ]
                ),
            ),
        ]
    )

    return pipe


def initialize_hist_forecaster():
    deseasonalizer_daily = Deseasonalizer(sp=24, model="additive")
    pipe = ForecastingPipeline(
        steps=[
            ("standardize", TabularToSeriesAdaptor(StandardScaler())),
            (
                "forecaster",
                TransformedTargetForecaster(
                    [
                        ("log_transformer", LogTransformer()),
                        # ("deseasonalizer_weekly", Deseasonalizer(sp=24*7, model="additive")),
                        (
                            "forecast",
                            make_reduction(
                                HistGradientBoostingRegressor(),
                                window_length=24,
                                strategy="direct",
                            ),
                        ),
                    ]
                ),
            ),
        ]
    )

    return pipe


from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor


def initialize_cat_forecaster():
    pipe = ForecastingPipeline(
        steps=[
            ("standardize", TabularToSeriesAdaptor(StandardScaler())),
            (
                "forecaster",
                TransformedTargetForecaster(
                    [
                        ("log_transformer", LogTransformer()),
                        (
                            "forecast",
                            make_reduction(
                                CatBoostRegressor(verbose=0, n_estimators=100),
                                window_length=24,
                                strategy="direct",
                            ),
                        ),
                    ]
                ),
            ),
        ]
    )

    return pipe


def initialize_lgbm_forecaster():
    pipe = ForecastingPipeline(
        steps=[
            ("standardize", TabularToSeriesAdaptor(StandardScaler())),
            (
                "forecaster",
                TransformedTargetForecaster(
                    [
                        ("log_transformer", LogTransformer()),
                        (
                            "forecast",
                            make_reduction(
                                LGBMRegressor(),
                                window_length=24,
                                strategy="direct",
                            ),
                        ),
                    ]
                ),
            ),
        ]
    )

    return pipe


def initialize_xgb_forecaster():
    pipe = ForecastingPipeline(
        steps=[
            ("standardize", TabularToSeriesAdaptor(StandardScaler())),
            (
                "forecaster",
                TransformedTargetForecaster(
                    [
                        ("log_transformer", LogTransformer()),
                        (
                            "forecast",
                            make_reduction(
                                XGBRegressor(objective="reg:squarederror"),
                                window_length=24,
                                strategy="direct",
                            ),
                        ),
                    ]
                ),
            ),
        ]
    )

    return pipe


def initialize_ets_forecaster():
    deseasonalizer_daily = Deseasonalizer(sp=24, model="additive")
    pipe = TransformedTargetForecaster(
        [
            ("log_transformer", LogTransformer()),
            # ("deseasonalizer_weekly", Deseasonalizer(sp=24*7, model="additive")),
            (
                "forecast",
                make_reduction(
                    AutoETS(auto=True, sp=24, n_jobs=-1),
                    window_length=24,
                    strategy="direct",
                ),
            ),
        ]
    )

In [4]:
arima_pipeline = initialize_arima_forecaster()
elasticnet_pipeline = initialize_elasticnet_forecaster()
rf_pipeline = initialize_rf_forecaster()
gb_pipeline = initialize_gb_forecaster()
hist_pipeline = initialize_hist_forecaster()
cat_pipeline = initialize_cat_forecaster()
lgbm_pipeline = initialize_lgbm_forecaster()
xgb_pipeline = initialize_xgb_forecaster()
ets_pipeline = initialize_ets_forecaster()


forecasting_models = {
    "elasticnet_pipeline": elasticnet_pipeline,
    "rf_pipeline": rf_pipeline,
    "gb_pipeline": gb_pipeline,
    "hist_pipeline": hist_pipeline,
    "ets_pipeline": ets_pipeline,
    "arima_pipeline": arima_pipeline,
    "cat_pipeline": cat_pipeline,
    "lgbm_pipeline": lgbm_pipeline,
    "xgb_pipeline": xgb_pipeline,
}

c:\Users\rkris\miniconda3\envs\slalomenv\Lib\site-packages\sktime\forecasting\compose\_reduce.py:1480: UserWarning: The `scitype` of the given `estimator` cannot be inferred. Assuming "tabular-regressor" = scikit-learn regressor interface. If this warning is followed by an unexpected exception, please consider report as a bug on the sktime issue tracker.
  warn(


In [5]:
selected_cols =  ['ail', 'gas_price', 'gas_tng', 'coal_tng',
       'wind_tng', 'solar_tng', 'hydro_tng', 'storage_tng', 'other_tng',
       'gas_avail', 'dual_fuel_avail', 'coal_avail', 'wind_avail',
       'solar_avail', 'hydro_avail', 'storage_avail', 'other_avail',
       'gas_reserve_margin', 'coal_reserve_margin', 'wind_reserve_margin',
       'solar_reserve_margin', 'hydro_reserve_margin',
       'storage_reserve_margin', 'other_reserve_margin', 'total_tng',
       'total_avail', 'gas_supply_mix', 'dual_fuel_supply_mix',
       'coal_supply_mix', 'wind_supply_mix', 'solar_supply_mix',
       'hydro_supply_mix', 'storage_supply_mix', 'other_supply_mix',
       'total_reserve_margin', 'relative_gas_reserve', 'demand_supply_ratio',
       'avail_gen_ratio', 'fossil_fuel_ratio']

In [6]:
sub_set_selected_cols =  ['ail', 'gas_price', 'gas_tng',
       'wind_tng', 'solar_tng', 'hydro_tng', 'wind_avail',
       'solar_avail', 'hydro_avail', 'storage_avail', 'other_avail',
       'gas_reserve_margin', 'wind_reserve_margin',
       'solar_reserve_margin', 'hydro_reserve_margin',
       'storage_reserve_margin', 'other_reserve_margin', 'total_tng',
       'total_avail', 'gas_supply_mix', 'wind_supply_mix', 
       'total_reserve_margin', 'demand_supply_ratio',
       'avail_gen_ratio', 'fossil_fuel_ratio']

subset_small = ['ail']

In [7]:
price_old_df = pd.read_csv(
    "https://raw.githubusercontent.com/slalom-ubc-mds/Power-Price-Prediction/main/data/processed/supply_load_price.csv",
    parse_dates=["Date (MST)"],
    index_col="Date (MST)",
)

# select only the month of January, 2022
price_old_df = price_old_df.loc["2023-01-01":"2023-01-31", subset_small+["price"]]

In [9]:
price_old_df = price_old_df.sort_values(by="Date (MST)")
price_old_df = price_old_df.asfreq("H")

y = price_old_df["price"]
X = price_old_df[subset_small]

In [30]:
cv = ExpandingWindowSplitter(
    initial_window=int(len(X) * 0.94), step_length=1, fh=np.arange(1, 13)
)

n_splits = cv.get_n_splits(y)
print(f"Number of Folds = {n_splits}")

Number of Folds = 34


In [33]:
# from sktime.utils.plotting import plot_windows, get_windows
# train_windows, test_windows = get_windows(y, cv)
# plot_windows(y, train_windows, test_windows)

In [48]:
from sktime.forecasting.model_evaluation import evaluate
from sktime.performance_metrics.forecasting import MeanSquaredScaledError, MeanSquaredError

list_models = ["elasticnet_pipeline", "rf_pipeline", "gb_pipeline", "hist_pipeline", "cat_pipeline", "lgbm_pipeline", "xgb_pipeline"]

rmse_cv_results = []
rmse_cv_std = []
for i in list_models:
    print(i)
    results = evaluate(
        forecaster=forecasting_models[i],
        y=y,
        X=X,
        cv=cv,
        strategy="refit",
        return_data=True,
        scoring=MeanSquaredError(square_root=True),
        backend="loky",
        error_score='raise'
    )
    
    rmse = results["test_MeanSquaredError"].mean()
    rmse_std = results["test_MeanSquaredError"].std()
    rmse_cv_results.append(rmse)
    rmse_cv_std.append(rmse_std)

elasticnet_pipeline
rf_pipeline
gb_pipeline
hist_pipeline
cat_pipeline
lgbm_pipeline
xgb_pipeline


In [49]:
rmse_cv_results_df = pd.DataFrame(
    {"Model": list_models, "RMSE_CV": rmse_cv_results, "RMSE_CV_STD": rmse_cv_std}
).sort_values(by=["RMSE_CV"])
rmse_cv_results_df

,Model,RMSE_CV,RMSE_CV_STD
0,elasticnet_pipeline,43.149016,14.476646
4,cat_pipeline,43.795019,16.037121
1,rf_pipeline,43.875671,14.703196
5,lgbm_pipeline,44.911613,17.147326
3,hist_pipeline,45.174434,17.086321
6,xgb_pipeline,45.787849,17.516343
2,gb_pipeline,46.889125,16.422389


In [50]:
price_old_df = pd.read_csv(
    "https://raw.githubusercontent.com/slalom-ubc-mds/Power-Price-Prediction/main/data/processed/supply_load_price.csv",
    parse_dates=["Date (MST)"],
    index_col="Date (MST)",
)

In [51]:
train = price_old_df.loc["2023-01-01":"2023-01-31", subset_small+["price"]]
test = price_old_df.loc["2023-02-01":"2023-02-02", subset_small+["price"]]
train = train.sort_values(by="Date (MST)")
train = train.asfreq("H")
test = test.sort_values(by="Date (MST)")
test = test.asfreq("H")
X_train = train[subset_small]
y_train = train["price"]
X_test = test[subset_small]
y_test = test["price"]

In [52]:
# fit and predict for all models
fh = np.arange(1, 13)
for i in list_models:
    print(i)
    forecasting_models[i].fit(y_train, X_train, fh=fh)
    y_pred = forecasting_models[i].predict(fh, X_train.tail(1))
    # get rmse between y_pred and y_test[:12]
    rmse = mean_squared_error(y_test[:12], y_pred, squared=False)
    print(rmse)

elasticnet_pipeline
31.92572748521333
rf_pipeline
28.94956001959079
gb_pipeline
28.886167907523856
hist_pipeline
39.873682918702166
cat_pipeline
11.687987042851185
lgbm_pipeline
27.136643860790493
xgb_pipeline
21.49555518196534


In [46]:
cv_results = evaluate(
        forecaster=forecasting_models['elasticnet_pipeline'],
        y=y,
        X=X,
        cv=cv,
        strategy="refit",
        return_data=True,
        scoring=MeanSquaredError(square_root=True),
        backend="loky",
        error_score='raise'
    )

In [45]:
import numpy as np
from sktime.performance_metrics.forecasting import MeanSquaredScaledError, MeanSquaredError
y_train = np.array([5, 0.5, 4, 6, 3, 5, 2])
y_true = np.array([3, -0.5, 100, 7, 2])
y_pred = np.array([2.5, 0.0, 2, 8, 1.25])
rmsse = MeanSquaredError(square_root=True)
rmsse(y_true, y_pred, y_train=y_train)

43.83163811677588

In [44]:
mean_squared_error(y_true, y_pred, squared=False)

43.83163811677588

In [ ]:

y_train = np.array([[0.5, 1], [-1, 1], [7, -6]])
y_true = np.array([[0.5, 1], [-1, 1], [7, -6]])
y_pred = np.array([[0, 2], [-1, 2], [8, -5]])
rmsse(y_true, y_pred, y_train=y_train)

rmsse = MeanSquaredScaledError(multioutput='raw_values', square_root=True)
rmsse(y_true, y_pred, y_train=y_train)

rmsse = MeanSquaredScaledError(multioutput=[0.3, 0.7], square_root=True)
rmsse(y_true, y_pred, y_train=y_train)

In [ ]:
# import pandas as pd

# def create_lagged_columns(X, lag_range=24):
#     lagged_names = []

#     for col in X:
#         for lag in range(1, lag_range + 1):
#             lagged_names.append(f"{col}_lag{lag}")

#     return lagged_names

# l = create_lagged_columns(['price'] + X.columns.values.tolist(), lag_range=24)

In [ ]:
# rf_pipeline.fit(y_train, X_train, fh=ForecastingHorizon(np.arange(1, 13)))

In [ ]:
# pd.DataFrame({'Coefficient': rf_pipeline.forecaster_.forecaster_.estimators_[0].feature_importances_, 'Label': l}).sort_values(by=['Coefficient'], ascending=False).head(10)

In [ ]:
# pd.DataFrame({'Coefficient': rf_pipeline.forecaster_.forecaster_.estimators_[1].feature_importances_, 'Label': l}).sort_values(by=['Coefficient'], ascending=False).head(10)

In [ ]:
# pd.DataFrame({'Coefficient': rf_pipeline.forecaster_.forecaster_.estimators_[2].feature_importances_, 'Label': l}).sort_values(by=['Coefficient'], ascending=False).head(10)

In [ ]:
# pd.DataFrame({'Coefficient': rf_pipeline.forecaster_.forecaster_.estimators_[3].feature_importances_, 'Label': l}).sort_values(by=['Coefficient'], ascending=False).head(10)

In [ ]:
# pd.DataFrame({'Coefficient': rf_pipeline.forecaster_.forecaster_.estimators_[4].feature_importances_, 'Label': l}).sort_values(by=['Coefficient'], ascending=False).head(10)

In [ ]:
# pd.DataFrame({'Coefficient': rf_pipeline.forecaster_.forecaster_.estimators_[5].feature_importances_, 'Label': l}).sort_values(by=['Coefficient'], ascending=False).head(10)

In [ ]:
# pd.DataFrame({'Coefficient': rf_pipeline.forecaster_.forecaster_.estimators_[6].feature_importances_, 'Label': l}).sort_values(by=['Coefficient'], ascending=False).head(10)

In [ ]:
# pd.DataFrame({'Coefficient': rf_pipeline.forecaster_.forecaster_.estimators_[7].feature_importances_, 'Label': l}).sort_values(by=['Coefficient'], ascending=False).head(10)

In [ ]:
# pd.DataFrame({'Coefficient': rf_pipeline.forecaster_.forecaster_.estimators_[8].feature_importances_, 'Label': l}).sort_values(by=['Coefficient'], ascending=False).head(10)

In [ ]:
# pd.DataFrame({'Coefficient': rf_pipeline.forecaster_.forecaster_.estimators_[9].feature_importances_, 'Label': l}).sort_values(by=['Coefficient'], ascending=False).head(10)

In [ ]:
# pd.DataFrame({'Coefficient': rf_pipeline.forecaster_.forecaster_.estimators_[10].feature_importances_, 'Label': l}).sort_values(by=['Coefficient'], ascending=False).head(10)

In [ ]:
# pd.DataFrame({'Coefficient': rf_pipeline.forecaster_.forecaster_.estimators_[11].feature_importances_, 'Label': l}).sort_values(by=['Coefficient'], ascending=False).head(10)

In [ ]:
# elasticnet_pipeline.fit(y_train, X_train, fh=ForecastingHorizon(np.arange(1, 13)))

In [ ]:
# from sklearn.inspection import permutation_importance
# # permutation_importance(elasticnet_pipeline, y_train, X_train, n_repeats=10, random_state=0)

In [ ]:
# elasticnet_pipeline._transform(y_train, X_train)

In [ ]:
# elasticnet_pipeline.forecaster_.get_params()


In [ ]:
# elasticnet_pipeline.forecaster_.forecaster_.get_fitted_params()


In [ ]:
# coef_df = pd.DataFrame({'Coefficient': elasticnet_pipeline.forecaster_.forecaster_.estimators_[0].coef_, 'Label': l})

In [ ]:
# print top 10 features
# coef_df.sort_values(by=['Coefficient'], ascending=False).head(10)

In [ ]:
# coef_df_2 = pd.DataFrame({'Coefficient': elasticnet_pipeline.forecaster_.forecaster_.estimators_[1].coef_, 'Label': l})
# coef_df_2.sort_values(by=['Coefficient'], ascending=False).head(10)

In [ ]:
# coef_df_2 = pd.DataFrame({'Coefficient': elasticnet_pipeline.forecaster_.forecaster_.estimators_[2].coef_, 'Label': l})
# coef_df_2.sort_values(by=['Coefficient'], ascending=False).head(10)